In [1]:
import sys, os
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_curve, roc_auc_score

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D

def load_joint_space_csv_chunks(file_path):
    data_frame = pd.read_csv(file_path, skiprows=1, header=None)
    del data_frame[18]
    return data_frame

def load_task_space_csv_chunks(file_path):
    return pd.read_csv(file_path, skiprows=1, header=None)


KeyboardInterrupt: 

In [ ]:


TRAIN_FOLDER = '/home/dfki.uni-bremen.de/bmanickavasakan/newdataset_rh5_leg/leg_5steps/'
TEST_FOLDER = '/home/dfki.uni-bremen.de/bmanickavasakan/newdataset_rh5_leg/leg_5steps/test_4steps'

X_TRAIN_FILE = os.path.join(TRAIN_FOLDER, 'leg_forwardkinematics_x.csv')
Q_TRAIN_FILE = os.path.join(TRAIN_FOLDER, 'leg_sysstate_q.csv')
x_train = load_task_space_csv_chunks(X_TRAIN_FILE)
q_train = load_joint_space_csv_chunks(Q_TRAIN_FILE)

X_TEST_FILE = os.path.join(TEST_FOLDER, 'leg_forwardkinematics_x.csv')
Q_TEST_FILE = os.path.join(TEST_FOLDER, 'leg_sysstate_q.csv')
x_test = load_task_space_csv_chunks(X_TEST_FILE)
q_test = load_joint_space_csv_chunks(Q_TEST_FILE)


In [ ]:
scaler = StandardScaler()
q_train = scaler.fit_transform(q_train)
q_test = scaler.fit_transform(q_test)

x_train_df = pd.DataFrame(x_train)
q_train_df = pd.DataFrame(q_train)
x_test_df = pd.DataFrame(x_test)
q_test_df = pd.DataFrame(q_test)

In [ ]:
print(q_train.shape)

In [ ]:
q_train_df.fillna(0, inplace=True)
to_analyze_columns = q_train_df.columns[0:17]

In [ ]:
clf = IsolationForest(n_estimators=100, max_samples='auto', max_features=1, bootstrap=False, n_jobs= -1, random_state=42, verbose=0)
clf.fit(q_train_df)

In [ ]:
pred = clf.predict(q_train_df)
q_train_df['anamoly'] = pred

outliers = q_train_df.loc[q_train_df['anamoly'] == -1]
outliers_index = list(outliers.index)

print(q_train_df['anamoly'].value_counts())

In [ ]:
print(q_train.mean())

In [ ]:

stats_q_train = pd.DataFrame()
stats_q_train["Mean"] = q_train_df.mean()
stats_q_train["Var"] = q_train_df.var()
stats_q_train["STD"] = q_train_df.std()
stats_q_train["OneSigmaMax"] = stats_q_train["Mean"] + stats_q_train["STD"]
stats_q_train["OneSigmaMin"] = stats_q_train["Mean"] - stats_q_train["STD"]
stats_q_train.T


max_std = stats_q_train["STD"].max()
colomn_max_std = stats_q_train["STD"].idxmax()

maximum = stats_q_train.loc[colomn_max_std, "Mean"] + (1.5 * max_std)
minimum = stats_q_train.loc[colomn_max_std, "Mean"] - (1.5 * max_std)

InDistribution_Q_Train = q_train_df[q_train_df[colomn_max_std].le(maximum) & q_train_df[colomn_max_std].ge(minimum)]
OutDistribution_Q_Train = q_train_df[q_train_df[colomn_max_std].ge(maximum) | q_train_df[colomn_max_std].le(minimum)]
test_df = pd.DataFrame()
test_df['anamoly'] = q_train_df[colomn_max_std].le(maximum) & q_train_df[colomn_max_std].ge(minimum)
#q_train.loc[q_train[colomn_max_std].ge(maximum) & q_train[colomn_max_std].le(minimum), 'anamoly'] = -1

print(test_df['anamoly'].value_counts())
print(test_df.shape)
print(q_train_df.shape)
outliers = q_train_df.loc[test_df['anamoly'] == False]
outliers_index = list(outliers.index)

print(outliers_index)

In [ ]:
pca = PCA(n_components=2)
pca.fit(q_train_df[to_analyze_columns])

res = pd.DataFrame(pca.transform(q_train_df[to_analyze_columns]))
Z = np.array(res)

In [ ]:
plt.title("Isolation Forest")
plt.contour(Z, cmap=plt.cm.Blues_r)

b1 = plt.scatter(res[0], res[1], c='green', s=20, label="normal points")
b1 = plt.scatter(res.iloc[outliers_index, 0], res.iloc[outliers_index, 1], c='red', s=20, label="predicted outliers", marker="X")
plt.legend(loc="upper right")
plt.show()
plt.savefig('StandardDev_2d_PCA.png')

In [ ]:
pca_nd = PCA(n_components=3)
q_scaler = q_train_df[to_analyze_columns]
q_reduce = pca_nd.fit_transform(q_scaler)


fig =plt.figure()
ax = fig.add_subplot(111, projection = '3d')
ax.set_zlabel("q_train composite 3")

ax.scatter(q_reduce[:, 0], q_reduce[:, 1], zs=q_reduce[:, 2], s=4, lw=1, label="normal points", c="green")

ax.scatter(q_reduce[outliers_index, 0], q_reduce[outliers_index, 1], q_reduce[outliers_index, 2], label="outliers", c="red", marker="x")
ax.legend()
plt.show()
plt.savefig('StandardDev_3d_PCA.png')


In [ ]:
pca_nd.explained_variance_ratio_

In [7]:
import tensorflow as tf
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices)

/device:GPU:0


AttributeError: module 'tensorflow' has no attribute 'config'

In [6]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

[[22. 28.]
 [49. 64.]]
